In [1]:
import numpy as np
import pandas as pd
import sys
from scipy import signal

nm = 1e-9
c = 2.99792458e8  # [m/s] Speed of light
hc = 1.987820871E-025  # [J * m / photon] Energy of photon with wavelength m

class Photonic:
	def __init__(self, config=None):
		self.config = config
		if config is None:
			self.config = 'Cfg1'

		# Read excel table having the following sheets: 'Light', 'Sensor', 'Scene', 'Lens', 'Op', 'Config'
		data_file = '../data/photonic_simul_data.xlsx'
		self.light_ = pd.read_excel(data_file,sheet_name='Light',header=1,index_col='Name')
		self.sensor_ = pd.read_excel(data_file,sheet_name='Sensor',header=1,index_col='Name')
		self.scene_ = pd.read_excel(data_file,sheet_name='Scene',header=1,index_col='Name')
		self.lens_ = pd.read_excel(data_file,sheet_name='Lens',header=1,index_col='Name')
		self.op_ = pd.read_excel(data_file,sheet_name='Op',header=1,index_col='Name')
		self.config_ = pd.read_excel(data_file,sheet_name='Config',header=1,index_col='Name')

		# check Excel data validity
		try:			
			# Reduce parameters level to those appear in the config, only.
			# print(' ## Photonic ## \n', config_.loc[self.cfg],'\n =====  \n' )
			self.light = self.light_.loc[self.config_.loc[self.config , 'Light']]
			self.scene = self.scene_.loc[self.config_.loc[self.config , 'Scene']]
			self.lens = self.lens_.loc[self.config_.loc[self.config , 'Lens']]
			self.sensor = self.sensor_.loc[self.config_.loc[self.config , 'Sensor']]
			self.op = self.op_.loc[self.config_.loc[self.config , 'Op']]
		except KeyError as err:
			print('\033[91m'+'&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
			print('Photonic::KeyError::Configutation \033[106m{}\033[0m\033[91m is mismatch key:\033[106m{}'.format(self.config, err)+ '\033[0m')
			sys.exit(1)

		self.wall_flux = self.wallFlux()
		self.silicon_flux = self.siliconFlux(self.wall_flux)

	def wallFlux(self, light=None, scene=None, dist_vec=None):
		if light is None:
			light = self.light
		if scene is None:
			scene = self.scene	
		if dist_vec is None:
			dist_m = scene.Distance_m
		else:
			dist_m = dist_vec

		# print(' ## wallFlux ## \n', light.PeakPower_W,'\n =====  \n' )
		# Flux on a wall (scene) during lighting time (pulse time)
		# Flux due to point source attached to the sensor
		return light.PeakPower_W * light.Transmission / (np.radians(light.Hfov_deg) * np.radians(light.Vfov_deg) * dist_m ** 2)  # [W/m^2]

	def siliconFlux(self, wall_flux, lens=None, scene=None):
		if lens is None:
			lens = self.lens
		if scene is None:
			scene = self.scene			
		# Flux on silicon from the scene during lighting time (pulse time)
		# Flux due to point source attached to the sensor
		# flux_active = light['peak'] * light['transmission'] / (light['hfov'] * light['vfov'] * scene['distance'] ** 2)  # [W/m^2]
		return wall_flux * scene.Reflectivity * lens.Transmission / (4 * lens.F_num ** 2)   # [W/m^2]

	def photoelectron(self, siliconFlux, sensor=None, light=None, op=None):
		if sensor is None:
			sensor = self.sensor
		if light is None:
			light = self.light
		if op is None:
			op = self.op
			
		energy_to_pe = hc / (light.WaveLength_nm * nm) # Conversion from energy [J] to number of photons
		# PE on silicon from the scene during lighting time (pulse time)
		pe_per_sec = siliconFlux * (sensor.PixelSize_m ** 2) * sensor.QE * sensor.FF / energy_to_pe  # [photoelectrons / sec] 
		pe_per_burst = pe_per_sec * op.InBurstDutyCycle * op.BurstTime_s  # [photoelectrons in burst]
		return pe_per_burst

	def siliconFlux2(self, light=None, scene=None, lens=None, dist_vec=None):
		return self.siliconFlux(wall_flux=self.wallFlux(dist_vec=dist_vec))

	def photoelectron2(self, light=None, scene=None, lens=None, sensor=None, op=None, dist_vec=None):
		return self.photoelectron(siliconFlux=self.siliconFlux(wall_flux=self.wallFlux(dist_vec=dist_vec)))

	def generate_pulse(self, delay=None, rise=None, fall=None, width=None, time_interval=None, smooth=False, mode='charge_discharge'):
		'''Create a light pulse (similar to a capacitor charge/discharge)
			rise - Pulse rise time [sec]
			fall - Pulse fall time [sec]
		    width - Pulse width time [sec]
		    time_interval - Sample time_interval [sec]
		'''
		self.time_interval = time_interval
		if time_interval is None:
			self.time_interval = 0.1e-9 # [sec]
		if delay is None:
			delay = 0.

		zeros_len = np.uint(10. + delay / self.time_interval)

		if mode == 'charge_discharge':
			x_rise = np.arange(0, width, self.time_interval)
			x_fall = np.arange(0, fall * 10, fall) # Discharge takes longer to become zero
			y_up = (1 - np.exp(-x_rise / rise))
			y_max = y_up.max()
			y_inf = (1 - np.exp(-(10. *rise) / rise)) # =1 @ infinity
			y_up = y_up / y_up.max()
			y_down = np.exp(-x_fall / fall)
			y_down = y_down / y_down.max()
			y_zeros = np.zeros(zeros_len)
		y = np.concatenate((y_zeros, y_up, y_down))

		# Smooth curves
		if smooth:
			f = signal.hamming(15)
			y = np.convolve(f, y, mode='same')
			y = y / y.max()
		y = y * y_max / y_inf

		# Clip negative 
		y[y < 0] = 0

		# Create time vector
		t = np.linspace(0, len(y) * self.time_interval, len(y))
		self.pulse_y = y
		self.pulse_t = t
		return y, t


	def conv_light_shutter(self, t_light=None, y_light=None, t_shutter=None, y_shutter=None, time_interval=None, ):
		self.time_interval = time_interval
		if time_interval is None:
			self.time_interval = 0.1e-9 # [sec]

		y = np.convolve(y_light, y_shutter, mode='full')
		y = y / y_light.sum() # Normalize convolution to the integrated light: y=1 if the entire illumination pulse is within the shutter
		t = np.linspace(0, len(y) * self.time_interval, len(y))
		return y, t

In [3]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

photonic = Photonic(config='Cfg3')

rise = 1e-14
fall = 1e-14
width = 1e-8
y1, t1 = photonic.generate_pulse(rise=rise, fall=fall, width=width, smooth=False)

rise = 1e-14
fall = 1e-14
width = 1e-8
delay = 3e-9
y2, t2 = photonic.generate_pulse(delay=delay, rise=rise, fall=fall, width=width, smooth=False)

y3, t3 = photonic.conv_light_shutter(t_light=t1, y_light=y1, t_shutter=t2, y_shutter=y2)

trace0 = go.Scatter(x=t1,
                    y=y1, mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='Light')
trace1 = go.Scatter(x=t2,
                    y=y2, mode='lines+markers',
                    name='Shutter')
trace2 = go.Scatter(x=t3-width-2*1e-9,
                    y=y3, mode='lines+markers',
                    name='Conv light-shutter')
trace3 = go.Scatter(x=[-13e-9],
                    y=[0.8], mode='text', textposition='top right',
                    name='text', text=['Convolution is normalized to the light integral' 
                                       + '<br>Equal 1.0 when light fully integrated by the shutter'])

data = [trace0, trace1, trace2, trace3]

layout = {'title': 'Light & Shutter pulses + its Convolution: both square and equal',
          'xaxis': {'title': 'time, time delay [sec]',
                    'type': 'linear'},  # Select 'log' or 'linear'
          'yaxis': {'title': 'Signal',
                    'type': 'linear'},  # Select 'log' or 'linear'
          'template': 'plotly_dark'}

iplot({'data': data, 'layout': layout})

In [4]:
photonic = Photonic(config='Cfg3')
rise = 1e-8
fall = 1e-8
width = 1e-8
y1, t1 = photonic.generate_pulse(rise=rise, fall=fall, width=width, smooth=False)

rise = 1e-14
fall = 1e-14
width = 0.8e-8
delay = 3e-9
y2, t2 = photonic.generate_pulse(delay=delay, rise=rise, fall=fall, width=width, smooth=False)

y3, t3 = photonic.conv_light_shutter(t_light=t1, y_light=y1, t_shutter=t2, y_shutter=y2)

trace0 = go.Scatter(x=t1,
                    y=y1, mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='Light')
trace1 = go.Scatter(x=t2,
                    y=y2, mode='lines+markers',
                    name='Shutter')
trace2 = go.Scatter(x=t3-width-2*1e-9,
                    y=y3, mode='lines+markers',
                    name='Conv light-shutter')
trace3 = go.Scatter(x=[-10e-9],
                    y=[0.8], mode='text', textposition='top right',
                    name='text', text=['Convolution is normalized to the light integral' 
                                       + '<br>Equal 1.0 when light fully integrated by the shutter'])

data = [trace0, trace1, trace2, trace3]

layout = {'title': 'Light & Shutter pulses + its Convolution: square shutter triangle light',
          'xaxis': {'title': 'time, time delay [sec]',
                    'type': 'linear'},  # Select 'log' or 'linear'
          'yaxis': {'title': 'Signal',
                    'type': 'linear'},  # Select 'log' or 'linear'
          'template': 'plotly_dark'}

iplot({'data': data, 'layout': layout})